In [23]:
# ev centrality, deg centrality, cred score, toxicity score to be considered for users
import pandas as pd

In [26]:
user_conspiracy = pd.read_csv("../Files/Tweets/result_cred_score_thresh_10.csv")
user_ev = pd.read_csv("../Files/networks/user_ev_centrality.csv")
user_deg = pd.read_csv("../Files/networks/user_deg_centrality.csv")
user_tox = pd.read_csv("../Files/user_toxicity.csv")

In [27]:
tweets = user_conspiracy.merge(user_ev, left_on="user_screen_name", right_on="name", how="left")
tweets = tweets[tweets["name"].notna()]
tweets = tweets[["HIGH","LOW","UNKW","SOCIAL","conspiracy_score","name","ev"]]
tweets = tweets.merge(user_deg, on="name", how="left")
tweets = tweets.merge(user_tox, left_on="name", right_on="user_screen_name", how="left")
tweets

,HIGH,LOW,UNKW,SOCIAL,conspiracy_score,name,ev,deg_cen,user_screen_name,toxicity_score
0,33,17267,171,0,1.0,infoitinterno,7.966932e-04,0.000324,infoitinterno,0.031807
1,0,5,0,13,1.0,Paolo90777,1.817897e-03,0.000139,Paolo90777,0.106862
2,0,7,3,1,1.0,ZiaGabriella1,5.881818e-03,0.000282,ZiaGabriella1,0.133203
3,0,3,1,32,1.0,lapillo1,2.029138e-03,0.000111,lapillo1,0.106676
4,0,33,3,0,1.0,terry57,9.011427e-05,0.000019,terry57,0.062098
...,...,...,...,...,...,...,...,...,...,...
8472,0,0,10,0,NaN,CloCecchetto,2.454355e-05,0.000102,NaN,NaN
8473,0,0,10,0,NaN,castelPao,4.582850e-06,0.000005,castelPao,0.068026
8474,0,0,10,0,NaN,traian_m,8.706800e-09,0.000014,NaN,NaN
8475,0,0,10,0,NaN,ComuneOristano,3.281069e-39,0.000005,NaN,NaN


In [21]:
# import numpy as np
# threshold_high_low = 20
# tweets = tweets[tweets['number_of_urls']>=threshold_high_low]
# tweets['conspiracy_score'] = np.around(((tweets['LOW'])/(tweets['HIGH']+tweets['LOW'])),2)
# tweets=tweets.sort_values(by=['conspiracy_score'], ascending=False)
# tweets.head(10)

In [28]:
tweets[["conspiracy_score","ev","deg_cen","toxicity_score"]].corr()

,conspiracy_score,ev,deg_cen,toxicity_score
conspiracy_score,1.000000,0.338556,0.024767,0.328570
ev,0.338556,1.000000,0.590345,0.274564
deg_cen,0.024767,0.590345,1.000000,0.060098
toxicity_score,0.328570,0.274564,0.060098,1.000000
